In [79]:
import os, sys
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import pickle
import logging
import mysql.connector
import time
import spacy
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from db_func import query_from_db

# Initialize drivers
ws_driver  = CkipWordSegmenter(level=3)
pos_driver = CkipPosTagger(level=3)
ner_driver = CkipNerChunker(level=3)


nlp = spacy.load("zh_core_web_trf")
os.environ['KMP_DUPLICATE_LIB_OK']= 'True'

### 1-1 Insert Developing Note

In [41]:

start = time.time()

logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)

with open(os.path.join(parent_dir, 'configs', 'loc2server.config'), 'rb') as f:
    configs = pickle.load(f)

mydb = mysql.connector.connect(
    host = configs['host'],
    user = configs['user'],
    passwd = configs['passwd'],
    database = configs['database'])

def insert_sentlevel_info(word_sentence: list, pos_sentence:list , news_sent_id, ws_approach, dep_sentence = ''):
    assert len(word_sentence) == len(pos_sentence)
    if ws_approach == 'ckip-transformer':
        for word_index, (word, word_pos) in enumerate(zip(word_sentence, pos_sentence)):
            word_cursor = mydb.cursor()
            try:
                word_cursor.execute("INSERT INTO news_db.news_words (news_sent_id, word_index, word, word_pos, ws_approach) VALUES (%s, %s, %s, %s, %s)", (news_sent_id, word_index, word, word_pos, ws_approach))
                #print("INSERT INTO news_db.news_words (news_sent_id, word_index, word, word_pos, ws_approach) VALUES ({}, {}, {}, {}, {})".format(news_sent_id, word_index, word, word_pos, ws_approach))

            except Exception as e:
                logging.error('Insert word pos error: {}\n News Sent ID: {}'.format(e, news_sent_id))
                print('Insert word pos error: {}\nNews Sent ID: {}\nWs_approach: {}'.format(e, news_sent_id, ws_approach))
                mydb.rollback()
            else:
                #print("else")
                mydb.commit()
            word_cursor.close()
    elif ws_approach == 'spacy-transformer':
        for word_index, (word, word_pos, word_dep) in enumerate(zip(word_sentence, pos_sentence, dep_sentence)):
            word_cursor = mydb.cursor()
            try:
                word_cursor.execute("INSERT INTO news_db.news_words (news_sent_id, word_index, word, word_pos, word_dep, ws_approach) VALUES (%s, %s, %s, %s, %s, %s)", (news_sent_id, word_index, word, word_pos, word_dep, ws_approach))
                #print("INSERT INTO news_db.news_words (news_sent_id, word_index, word, word_pos, word_dep, ws_approach) VALUES ({}, {}, {}, {}, {}, {})".format(news_sent_id, word_index, word, word_pos, word_dep, ws_approach))

            except Exception as e:
                logging.error('Insert word pos error: {}\n News Sent ID: {}'.format(e, news_sent_id))
                print('Insert word pos error: {}\nNews Sent ID: {}\nWs_approach: {}'.format(e, news_sent_id, ws_approach))
                mydb.rollback()
            else:
                mydb.commit()
            word_cursor.close()
    else:
        logging.error("Error ws_approach")
        sys.exit(0)

def insert_ner_info(entity_sent_list, news_sent_id, ner_approach):
    for ent_id, (ent_text, ent_type, ent_index) in enumerate(sorted(entity_sent_list, key = lambda x:x[2][0])):
        ner_cursor = mydb.cursor()
        try:
            ner_cursor.execute("INSERT INTO news_db.news_ners (news_sent_id, start_index, end_index, ent_type, ent_text, ner_approach) VALUES (%s, %s, %s, %s, %s, %s)", (news_sent_id, ent_index[0], ent_index[1], ent_type, ent_text, ner_approach))
            #print("INSERT INTO news_db.news_ners (news_sent_id, start_index, end_index, ent_type, ent_text, ner_approach) VALUES ({}, {}, {}, {}, {}, {})".format(news_sent_id, ent_index[0], ent_index[1], ent_type, ent_text, ner_approach))
        
        except Exception as e:
            logging.error('Insert NER error: {}\n News Sent ID: {}'.format(e, news_sent_id))
            print('Insert NER error: {}\n News Sent ID: {}'.format(e, news_sent_id))
            mydb.rollback()
        else:
            mydb.commit()
        ner_cursor.close()
        
def insert_process_flag(news_sent_id, process_name):
    process_cursor = mydb.cursor()
    try:
        process_cursor.execute("INSERT INTO news_db.sent_processes (news_sent_id, process_name) VALUES (%s, %s)", (news_sent_id, process_name))
        #print("INSERT INTO news_db.nlp_processes (news_sent_id, process_name) VALUES ({}, {})".format(news_sent_id, process_name))
    except Exception as e:
        logging.error('Process Insert Error: {}\nNews sent ID: {}'.format(e, news_sent_id))
        #print('Process Insert Error: {}\nNews sent ID: {}'.format(e, news_sent_id))
        mydb.rollback()
    else:
        mydb.commit()
    process_cursor.close()

def sent_level_analysis(raw_df):
    for index, (content_sent_id, sent) in raw_df.iterrows():
        insert_process_flag(content_sent_id, 'sent-analysis')
        try:
            word_sentence_list  = ws_driver(text, use_delim=False)
            entity_sentence_list = ner_driver(text, use_delim=False)
            pos_sentence_list = pos_driver(word_sentence_list, use_delim=False)
            spacy_doc = nlp(sent)
            word_sent_list_spacy, word_pos_list_spacy, word_dep_list_spacy = zip(*[(token.text, token.tag_, token.dep_) for token in spacy_doc])
            entity_sent_list_spacy = [(ent.text, ent.label_, (ent.start_char, ent.end_char)) for ent in spacy_doc.ents]
        except Exception as e:
            logging.error('NLP process Error: {}\n Content ID: {}'.format(e, content_sent_id))
            print('NLP process Error: {}\n Content ID: {}'.format(e, content_sent_id))
        
        
        insert_sentlevel_info(word_sentence_list[0],  pos_sentence_list[0], content_sent_id, 'ckip-transformer')
        insert_sentlevel_info(word_sent_list_spacy, word_pos_list_spacy, content_sent_id, 'spacy-transformer', word_dep_list_spacy)
        insert_ner_info(entity_sentence_list[0], content_sent_id, 'ckip-transformer')
        insert_ner_info(entity_sent_list_spacy, content_sent_id, 'spacy-transformer')
     

raw_df = query_from_db("SELECT nns.news_sent_id, nns.sent FROM news_db.news_sents nns WHERE not exists (SELECT 1 FROM news_db.sent_processes sp WHERE sp.process_name = \'sent-analysis\' and sp.news_sent_id = nns.news_sent_id) LIMIT 100")
sent_level_analysis(raw_df)
mydb.close()
logging.error('Finish process {} examples in {} seconds'.format(len(raw_df), time.time() - start))
print('Finish process {} examples in {} seconds'.format(len(raw_df), time.time() - start))

Inference: 100%|██████████| 1/1 [00:00<00:00,  3.68it/s]
2021-02-19 22:26:58,725: ERROR: Finish process 100 examples in 523.4907739162445 seconds


Finish process 100 examples in 523.4921119213104 seconds


### Bulk insert developing 

In [106]:
raw_df = query_from_db("SELECT * FROM news_db.financial_sent_view LIMIT 16;")

In [99]:
[1, 2, 3][:5]

[1, 2, 3]

In [118]:
def sent_generator(id_lst, sent_lst, batch_size):
    current_idx = 0
    max_len = len(sent_lst)
    while True:
        yield  id_lst[current_idx:current_idx + batch_size], sent_lst[current_idx:current_idx + batch_size]
        current_idx += batch_size
        if current_idx >= max_len:
            break
"INSERT INTO news_db.news_words (news_sent_id, word_index, word, word_pos, word_dep, ws_approach) VALUES (%s, %s, %s, %s, %s, %s)"
"INSERT INTO news_db.news_ners (news_sent_id, start_index, end_index, ent_type, ent_text, ner_approach) VALUES (%s, %s, %s, %s, %s, %s)"
news_words_res = []
news_ners_res = []
for sent_ids, sents in sent_generator(raw_df['news_sent_id'].tolist(), raw_df['sent'].tolist(), 4):
    word_sentence_list  = ws_driver(sents, use_delim=False)
    entity_sentence_list = ner_driver(sents, use_delim=False)
    pos_sentence_list = pos_driver(word_sentence_list, use_delim=False)
    for sent_id, word_res, pos_res in zip(sent_ids, word_sentence_list, pos_sentence_list):
        for index, (word, word_pos) in enumerate(zip(word_res, pos_res)):
            news_words_res.append((sent_id, index, word, word_pos, None, 'ckip-transformer'))
    for sent_id, entities in zip(sent_ids, entity_sentence_list):
        for ent_text, ent_type, ent_index in sorted(entities, key = lambda x:x[2][0]):
            news_ners_res.append((sent_id, ent_index[0], ent_index[1], ent_type, ent_text, 'ckip-transformer'))

Inference: 100%|██████████| 1/1 [00:00<00:00,  1.02it/s]


In [119]:
news_words_res

[(262697, 0, '拜登', 'Nc', None, 'ckip-transformer'),
 (262697, 1, '政府', 'Na', None, 'ckip-transformer'),
 (262697, 2, '官員', 'Na', None, 'ckip-transformer'),
 (262697, 3, '承認', 'VE', None, 'ckip-transformer'),
 (262697, 4, '，', 'COMMACATEGORY', None, 'ckip-transformer'),
 (262697, 5, '眼前', 'Nc', None, 'ckip-transformer'),
 (262697, 6, '有', 'V_2', None, 'ckip-transformer'),
 (262697, 7, '許多', 'Neqa', None, 'ckip-transformer'),
 (262697, 8, '硬仗', 'Na', None, 'ckip-transformer'),
 (262697, 9, '要', 'D', None, 'ckip-transformer'),
 (262697, 10, '打', 'VC', None, 'ckip-transformer'),
 (262697, 11, '。', 'PERIODCATEGORY', None, 'ckip-transformer'),
 (262697, 12, '應變', 'VA', None, 'ckip-transformer'),
 (262697, 13, '之', 'DE', None, 'ckip-transformer'),
 (262697, 14, '道', 'Na', None, 'ckip-transformer'),
 (262697, 15, '之', 'DE', None, 'ckip-transformer'),
 (262697, 16, '一', 'Neu', None, 'ckip-transformer'),
 (262697, 17, '是', 'SHI', None, 'ckip-transformer'),
 (262697, 18, '透過', 'P', None, 'ckip-tr

In [ ]:
# Initialize drivers
ws_driver  = CkipWordSegmenter(level=3)
pos_driver = CkipPosTagger(level=3)
ner_driver = CkipNerChunker(level=3)

In [1]:
text = ["""中國大陸23日共新增80例2019冠狀病毒疾病（COVID-19）確診，其中65例為本土病例。首都北京和經濟大城上海仍不斷有疫情，各新增2例、3例本土確診。"""]

In [ ]:
ner

In [4]:
import logging
import os, sys
import time
import re
import pickle
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

from db_func import query_from_db

In [5]:
raw_df = query_from_db("""SELECT nns.news_sent_id, nns.sent FROM news_db.news_sents nns 
WHERE NOT EXISTS (SELECT 1 FROM news_db.sent_processes sp WHERE sp.process_name = 'sent-analysis' and sp.news_sent_id = nns.news_sent_id) LIMIT 100""")

In [6]:
import logging
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)


In [5]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']= 'True'

from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
# Initialize drivers
ws_driver  = CkipWordSegmenter(level=3)
pos_driver = CkipPosTagger(level=3)
ner_driver = CkipNerChunker(level=3)

In [7]:
for index, (sent_id, sent) in raw_df.iterrows():
    # Enable sentence segmentation
    ws  = ws_driver([sent], use_delim=False)
    ner = ner_driver([sent], use_delim=False)

    # Disable sentence segmentation
    pos = pos_driver(ws, use_delim=False)
    print('Sent: ', sent)
    print(ws, pos, ner)

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Sent:  文化部長李永得日前要求審查陸書，讓出版業相當不滿，而事後李永得也找來業者開會尋求配套，不過有業者不滿李永得疑似玩兩面手法，因為開會當下李永得是允諾形式審查，不過事後接受專訪卻又說「反對審查的業者，根本是在台灣，幫中國強權維護他們的威權」，對於質疑，李永得回應是外界抹黑！
[['文化部長', '李永得', '日前', '要求', '審查', '陸書', '，', '讓', '出版業', '相當', '不滿', '，', '而', '事', '後', '李永得', '也', '找來', '業者', '開會', '尋求', '配套', '，', '不過', '有', '業者', '不滿', '李永得', '疑似', '玩', '兩', '面', '手法', '，', '因為', '開會', '當下', '李永得', '是', '允諾', '形式', '審查', '，', '不過', '事', '後', '接受', '專訪', '卻', '又', '說', '「', '反對', '審查', '的', '業者', '，', '根本', '是', '在', '台灣', '，', '幫', '中國', '強權', '維護', '他們', '的', '威權', '」', '，', '對於', '質疑', '，', '李永得', '回應', '是', '外界', '抹黑', '！']] [['Na', 'Nb', 'Nd', 'VF', 'VC', 'Na', 'COMMACATEGORY', 'VL', 'Na', 'Dfa', 'VK', 'COMMACATEGORY', 'Cbb', 'Na', 'Ng', 'Nb', 'D', 'VC', 'Na', 'VA', 'VC', 'VA', 'COMMACATEGORY', 'Cbb', 'V_2', 'Na', 'VK', 'Nb', 'VG', 'VC', 'Neu', 'Na', 'Na', 'COMMACATEGORY', 'Cbb', 'VA', 'D', 'Nb', 'SHI', 'VE', 'Na', 'VC', 'COMMACATEGORY', 'Cbb', 'Na', 'Ng', 'VC', 'VC', 'D', 'D', 'VE', 'PARENTHESISCATEGORY', 'VE', 'VC

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Sent:  台灣文學基地開幕，部長李永得碰上文資團體來陳情，只是部長允諾好好處理的同時，出版業的聲音，是否也能比照辦理，因為李永得要求審查陸書，讓出版業覺得言論自由，被限制！
[['台灣', '文學', '基地', '開幕', '，', '部長', '李永得', '碰上', '文資', '團體', '來', '陳情', '，', '只是', '部長', '允諾', '好好', '處理', '的', '同時', '，', '出版業', '的', '聲音', '，', '是否', '也', '能', '比照', '辦理', '，', '因為', '李永得', '要求', '審查', '陸書', '，', '讓', '出版業', '覺得', '言論', '自由', '，', '被', '限制', '！']] [['Nc', 'Na', 'Nc', 'VH', 'COMMACATEGORY', 'Na', 'Nb', 'VC', 'Na', 'Na', 'D', 'VF', 'COMMACATEGORY', 'Cbb', 'Na', 'VE', 'D', 'VC', 'DE', 'Nd', 'COMMACATEGORY', 'Na', 'DE', 'Na', 'COMMACATEGORY', 'D', 'D', 'D', 'P', 'VC', 'COMMACATEGORY', 'Cbb', 'Nb', 'VF', 'VC', 'Na', 'COMMACATEGORY', 'VL', 'Na', 'VK', 'Na', 'VH', 'COMMACATEGORY', 'P', 'VE', 'EXCLAMATIONCATEGORY']] [[NerToken(word='台灣文學基地', ner='FAC', idx=(0, 6)), NerToken(word='李永得', ner='PERSON', idx=(11, 14)), NerToken(word='李永得', ner='PERSON', idx=(56, 59))]]


Inference:   0%|          | 0/1 [00:00<?, ?it/s]

...

Sent:  前瞻2.0經費約10億元，補助對象分別是桃園中正公園、台南永康、台南北區以及桃園區文化局，這樣算下來10億元用得差不多，新北的永平國小、板橋海山高中、永平立體、三重玫瑰停車場，順列排在第5、6、7、9但明明這幾個地方也有不少人居住，究竟核發標準是甚麼？
[['前瞻', '2.0', '經費', '約', '10億', '元', '，', '補助', '對象', '分別', '是', '桃園', '中正', '公園', '、', '台南', '永康', '、', '台南', '北區', '以及', '桃園區', '文化局', '，', '這樣', '算下來', '10億', '元', '用', '得', '差不多', '，', '新北', '的', '永平', '國小', '、', '板橋', '海山', '高中', '、', '永平', '立體', '、', '三重', '玫瑰', '停車場', '，', '順列', '排', '在', '第5', '、', '6', '、', '7', '、', '9', '但', '明明', '這', '幾', '個', '地方', '也', '有', '不少', '人', '居住', '，', '究竟', '核發', '標準', '是', '甚麼', '？']] [['VC', 'Neu', 'Na', 'Da', 'Neu', 'Nf', 'COMMACATEGORY', 'VD', 'Na', 'D', 'SHI', 'Nc', 'Nb', 'Nc', 'PAUSECATEGORY', 'Nc', 'Nc', 'PAUSECATEGORY', 'Nc', 'Nc', 'Caa', 'Nc', 'Nc', 'COMMACATEGORY', 'VH', 'VG', 'Neu', 'Nf', 'VC', 'DE', 'VH', 'COMMACATEGORY', 'Nc', 'DE', 'Nc', 'Nc', 'PAUSECATEGORY', 'Nc', 'Nc', 'Nc', 'PAUSECATEGORY', 'Nc', 'VH', 'PAUSECATEGORY', 'Nc', 'Na', 'Nc', 'COMMACATEGORY', 'D', 'VG', 'P', 'Neu', 'PAUSECATEGORY', 

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Sent:  立委(國)洪孟楷：「新北市是403萬人口，也是人口數最多的一個直轄市，卻被排在5、6、7、8(9)名，這難免會讓大家覺得說，奇怪這個分配上面是不是有所不均。」
[['立委', '(', '國', ')', '洪孟楷', '：', '「', '新北市', '是', '403萬', '人口', '，', '也', '是', '人口數', '最多', '的', '一', '個', '直轄市', '，', '卻', '被', '排', '在', '5', '、', '6', '、', '7', '、', '8', '(', '9)', '名', '，', '這', '難免', '會', '讓', '大家', '覺得', '說', '，', '奇怪', '這', '個', '分配', '上面', '是', '不', '是', '有所', '不均', '。', '」']] [['Na', 'PARENTHESISCATEGORY', 'Nc', 'PARENTHESISCATEGORY', 'Nb', 'COLONCATEGORY', 'PARENTHESISCATEGORY', 'Nc', 'SHI', 'Neu', 'Na', 'COMMACATEGORY', 'D', 'SHI', 'Na', 'VH', 'DE', 'Neu', 'Nf', 'Na', 'COMMACATEGORY', 'D', 'P', 'VC', 'P', 'Neu', 'PAUSECATEGORY', 'Neu', 'PAUSECATEGORY', 'Neu', 'PAUSECATEGORY', 'Neu', 'PARENTHESISCATEGORY', 'Neu', 'Nf', 'COMMACATEGORY', 'Nep', 'D', 'D', 'VL', 'Nh', 'VK', 'VE', 'COMMACATEGORY', 'VK', 'Nep', 'Nf', 'VD', 'Ncd', 'SHI', 'D', 'SHI', 'VJ', 'VH', 'PERIODCATEGORY', 'PARENTHESISCATEGORY']] [[NerToken(word='洪孟楷', ner='PERSON', idx=(5, 8)), NerToken(word='新北市', ner='

Inference:   0%|          | 0/1 [00:00<?, ?it/s]


KeyboardInterrupt: 

In [6]:
ws  = ws_driver(text, use_delim=False)
ner = ner_driver(text, use_delim=False)

# Disable sentence segmentation
pos = pos_driver(ws, use_delim=False)

Inference: 100%|██████████| 1/1 [00:00<00:00,  2.60it/s]


In [21]:
for word, ner[0])

[NerToken(word='中國大陸', ner='GPE', idx=(0, 4)), NerToken(word='23日', ner='DATE', idx=(4, 7)), NerToken(word='80', ner='CARDINAL', idx=(10, 12)), NerToken(word='2019冠狀病毒疾病', ner='EVENT', idx=(13, 23)), NerToken(word='65', ner='CARDINAL', idx=(38, 40)), NerToken(word='北京', ner='GPE', idx=(49, 51)), NerToken(word='上海', ner='GPE', idx=(56, 58)), NerToken(word='2', ner='CARDINAL', idx=(68, 69)), NerToken(word='3', ner='CARDINAL', idx=(71, 72))]


In [3]:
ws_driver(text, use_delim=False)

Inference: 100%|██████████| 1/1 [00:00<00:00,  2.71it/s]


[['中國',
  '大陸',
  '23日',
  '共',
  '新增',
  '80',
  '例',
  '2019',
  '冠狀病毒',
  '疾病',
  '（',
  'COVID-19',
  '）',
  '確診',
  '，',
  '其中',
  '65',
  '例',
  '為',
  '本土',
  '病例',
  '。',
  '首都',
  '北京',
  '和',
  '經濟',
  '大城',
  '上海',
  '仍',
  '不斷',
  '有',
  '疫情',
  '，',
  '各',
  '新增',
  '2',
  '例',
  '、',
  '3',
  '例',
  '本土',
  '確診',
  '。']]

In [2]:
import os

os.environ['KMP_DUPLICATE_LIB_OK']='True'
import spacy
nlp = spacy.load("zh_core_web_trf")

In [31]:
for ent in nlp(text[0]).ents:
    print(ent.text, ent.label_)

中國大陸 GPE
23日 DATE
80 CARDINAL
2019冠狀病毒疾病 EVENT
65 CARDINAL
北京 GPE
上海 GPE
2例 CARDINAL
3例 CARDINAL


In [33]:
for token in nlp(text[0]):
    print(token, token.dep_, token.tag_)

中國 nmod:assmod NR
大陸 nsubj NN
23日 nmod:tmod NT
共 advmod AD
新增 ROOT VV
80 nummod CD
例 mark:clf M
2019 compound:nn NT
冠狀 compound:nn NN
病毒 compound:nn NN
疾病 dep NN
（ punct PU
COVID parataxis:prnmod PU
- dobj NR
19 parataxis:prnmod NT
） punct PU
確診 ccomp VV
， punct PU
其中 dep NN
65 dep CD
例為 cop M
本土 compound:nn NN
病例 conj NN
。 punct PU
首都 appos NN
北京 conj NR
和 cc CC
經濟 compound:nn NN
大城 appos NN
上海 nsubj NR
仍 advmod AD
不 advmod AD
斷 advmod AD
有 ROOT VE
疫情 dobj NN
， punct PU
各 advmod AD
新增 conj VV
2例 dep M
、 punct PU
3例 mark:clf M
本土 dep NN
確診 dobj VV
。 punct PU


In [46]:
source_list = """
'ETtoday','財經新聞'
'PCHOME','財經'
'Yahoo奇摩新聞','財經新聞'
'Yahoo奇摩新聞','運動新聞'
'Yahoo奇摩股市','台股動態'
'Yahoo奇摩股市','國際財經'
'Yahoo奇摩股市','基金動態'
'Yahoo奇摩股市','專家專欄'
'Yahoo奇摩股市','小資理財'
'Yahoo奇摩股市','最新股市'
'Yahoo奇摩股市','研究報導'
'中央社即時新聞','財經新聞'
'台視新聞','財經'
'新浪台灣新聞中心','財經'
'新頭殼要聞','財經'
'民報 Taiwan People News','財經新聞'
'經濟日報','2021全球名家瞭望'
'經濟日報','2021投資前瞻'
'經濟日報','2021新春財經特輯'
'經濟日報','ECFA十周年'
'經濟日報','ESG'
'經濟日報','Fintech'
'經濟日報','Focus |防疫國家隊'
'經濟日報','ICT趨勢'
'經濟日報','K型復甦'
'經濟日報','一分鐘看世界'
'經濟日報','上市櫃公司熱門股排行'
'經濟日報','五金新訊'
'經濟日報','企業CEO'
'經濟日報','個人理財'
'經濟日報','健康元氣'
'經濟日報','光電半導體'
'經濟日報','兩岸快遞'
'經濟日報','兩岸焦點'
'經濟日報','公司治理'
'經濟日報','創業之星'
'經濟日報','化工科技'
'經濟日報','十四五突圍'
'經濟日報','台中韓大戰'
'經濟日報','品味生活'
'經濟日報','國內共同基金淨值'
'經濟日報','國際期貨'
'經濟日報','國際焦點'
'經濟日報','國際現場'
'經濟日報','基金天地'
'經濟日報','外匯市場'
'經濟日報','大數字'
'經濟日報','就市論勢'
'經濟日報','工具機'
'經濟日報','市場焦點'
'經濟日報','幸福城市大調查'
'經濟日報','建材新訊'
'經濟日報','建案開箱'
'經濟日報','房市情報'
'經濟日報','房市焦點'
'經濟日報','房市點線面'
'經濟日報','房產投資'
'經濟日報','投行看大陸'
'經濟日報','投資報告'
'經濟日報','指標焦點股'
'經濟日報','政經焦點'
'經濟日報','新冠肺炎防疫'
'經濟日報','新南向專題'
'經濟日報','日經中文網'
'經濟日報','時尚生活'
'經濟日報','智慧製造'
'經濟日報','智能股市'
'經濟日報','期貨商論壇'
'經濟日報','期貨市場'
'經濟日報','樂活旅遊'
'經濟日報','櫃買市場'
'經濟日報','權證特區'
'經濟日報','消費生活'
'經濟日報','熱門亮點'
'經濟日報','熱門話題'
'經濟日報','理財部落客'
'經濟日報','生技醫藥'
'經濟日報','產學研訓'
'經濟日報','產業動態'
'經濟日報','產業熱點'
'經濟日報','產業達人'
'經濟日報','社論'
'經濟日報','科技新視野'
'經濟日報','稅務法規'
'經濟日報','精品時尚'
'經濟日報','經濟周報'
'經濟日報','經營管理'
'經濟日報','綠色產業'
'經濟日報','總經趨勢'
'經濟日報','總編推薦'
'經濟日報','美中貿易戰'
'經濟日報','美國總統大選'
'經濟日報','職場風向'
'經濟日報','自動化產業'
'經濟日報','英語大進化'
'經濟日報','論壇'
'經濟日報','財富管理'
'經濟日報','責任投資／企業永續'
'經濟日報','資訊圖表'
'經濟日報','退休理財'
'經濟日報','運動休旅'
'經濟日報','醫藥生技'
'經濟日報','金融脈動'
'經濟日報','銀行保險'
'經濟日報','陸港行情'
'經濟日報','集中市場'
'經濟日報','願景工程'
'經濟日報','風格人物'
'經濟日報','食品餐飲'
'經濟日報','食尚饗宴'
'自由時報','財經新聞'
'風傳媒','財經'
""".split('\n')

In [65]:
source_category_lst = []
for lst in source_list:
    if lst:
        source_category_lst.append([element.strip("'") for element in lst.split(',')])

In [77]:
res = ''
for source, category in source_category_lst:
    res += "OR (nrf.news_source = '{}' AND nrf.news_category = '{}')".format(source, category)

In [78]:
res

"OR (nrf.news_source = 'ETtoday' AND nrf.news_category = '財經新聞')OR (nrf.news_source = 'PCHOME' AND nrf.news_category = '財經')OR (nrf.news_source = 'Yahoo奇摩新聞' AND nrf.news_category = '財經新聞')OR (nrf.news_source = 'Yahoo奇摩新聞' AND nrf.news_category = '運動新聞')OR (nrf.news_source = 'Yahoo奇摩股市' AND nrf.news_category = '台股動態')OR (nrf.news_source = 'Yahoo奇摩股市' AND nrf.news_category = '國際財經')OR (nrf.news_source = 'Yahoo奇摩股市' AND nrf.news_category = '基金動態')OR (nrf.news_source = 'Yahoo奇摩股市' AND nrf.news_category = '專家專欄')OR (nrf.news_source = 'Yahoo奇摩股市' AND nrf.news_category = '小資理財')OR (nrf.news_source = 'Yahoo奇摩股市' AND nrf.news_category = '最新股市')OR (nrf.news_source = 'Yahoo奇摩股市' AND nrf.news_category = '研究報導')OR (nrf.news_source = '中央社即時新聞' AND nrf.news_category = '財經新聞')OR (nrf.news_source = '台視新聞' AND nrf.news_category = '財經')OR (nrf.news_source = '新浪台灣新聞中心' AND nrf.news_category = '財經')OR (nrf.news_source = '新頭殼要聞' AND nrf.news_category = '財經')OR (nrf.news_source = '民報 Taiwan People News' AND

In [76]:
"""記者施怡妏／綜合報導 台積電（2330）先前衝破600元大關後不斷創新高，令不少投資人心動跟著上車，甚至不惜高價買進，怎料近期卻接連下跌，有男網友當時看到股價狂飆心癢癢，在台積電歷史新高679元時花136萬買了兩張，沒想到之後股價竟然一路下跌，他因此感到很苦惱，「大概什麼時候可以解套呢？」 男網友在Dcard上發文，在台積電歷史新高679元時買進兩張，大概花了136萬元，但股價卻沒有繼續漲，「如果把台積電發的股利也算進去，每三個月能領5000元」，想知道大概要過多久才可以解套，「能把賣掉兩張台積電的錢，加上台積電兩張的股利收入，總金額大於成本136萬元」。 貼文一出後，不少人驚訝表示「怎麼那麼剛好買在最高點」，「你怎麼可能剛好買在歷史最高點，679元買兩張，你是神嗎」、「下次要買麻煩告知一下，謝謝，我想賣在最高點」、「知道股票解套時間的早就賺翻了，還在這邊回你」、「股神吧，買在最高點欸」。 有人建議他直接賣掉停損，「勸你現在下車，現在籌碼太亂了出來觀望比較好」、「上來問就是沒信心抱住啦，建議你認賠換現金好過年，重新省思策略」。也有人建議他可以繼續放著，「買在466元的那時候很多人說會套十年，結果才過幾個月就破600，466一解套就賣掉的現在還在擦眼淚」、「大家都說上看1000，你可以慢慢等」、「不缺錢哦的話就繼續放著」。"""

'記者施怡妏／綜合報導 台積電（2330）先前衝破600元大關後不斷創新高，令不少投資人心動跟著上車，甚至不惜高價買進，怎料近期卻接連下跌，有男網友當時看到股價狂飆心癢癢，在台積電歷史新高679元時花136萬買了兩張，沒想到之後股價竟然一路下跌，他因此感到很苦惱，「大概什麼時候可以解套呢？」 男網友在Dcard上發文，在台積電歷史新高679元時買進兩張，大概花了136萬元，但股價卻沒有繼續漲，「如果把台積電發的股利也算進去，每三個月能領5000元」，想知道大概要過多久才可以解套，「能把賣掉兩張台積電的錢，加上台積電兩張的股利收入，總金額大於成本136萬元」。 貼文一出後，不少人驚訝表示「怎麼那麼剛好買在最高點」，「你怎麼可能剛好買在歷史最高點，679元買兩張，你是神嗎」、「下次要買麻煩告知一下，謝謝，我想賣在最高點」、「知道股票解套時間的早就賺翻了，還在這邊回你」、「股神吧，買在最高點欸」。 有人建議他直接賣掉停損，「勸你現在下車，現在籌碼太亂了出來觀望比較好」、「上來問就是沒信心抱住啦，建議你認賠換現金好過年，重新省思策略」。也有人建議他可以繼續放著，「買在466元的那時候很多人說會套十年，結果才過幾個月就破600，466一解套就賣掉的現在還在擦眼淚」、「大家都說上看1000，你可以慢慢等」、「不缺錢哦的話就繼續放著」。'

In [134]:
import pytz
from datetime import datetime, timedelta
str(datetime.now(tz = pytz.timezone('Asia/Taipei')).date() - timedelta(days = 7))

'2021-02-19'

In [133]:
datetime.now()

datetime.timedelta(days=7)